<a href="https://colab.research.google.com/github/Wozny614/mispy/blob/master/Minimalizacja_mocy_Dariusz_Wo%C5%BAnica_215369.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp

In [2]:
from pulp import *

# Maksymalny Prąd

               |---R1---|        |---R4---|
     ----------|        |---R3---|        |----------
               |---R2---|        |---R5---|


 Dla takiego układu rezystorów, znając ich rezystancje oraz maksymalny prąd, który może przez nie przepłynąć należy obliczyć maksymalny prąd który może przepłynąć przez cały układ.


|Rezystor|R1|R2|R3|R4|R5
|--      |--|--|--|--|--|
|R       |8|6|4|10|8|
|Imax    |2|3|4|2 |2|


Do tego problemu wykorzystana została zależność wynikająca z prawa Kirchoffa, mówiąca, że suma prądów wchodzących do węzła jest równa sumie prądów wychodzących z tego węzła.

Zmiennymi decyzyjnymi są prądy płynące przez poszczególne rezystory, przy czym muszą zostać zachowane zależności wynikające z powyższego prawa Kirchoffa, gdzie jeden prąd oddziałuje na pozostałe.

Wykorzystując suwaki można sprawdzić wyniki analizy dla różnych rezystancji, oraz różnych maksymalnych przez nie płynących prądów.



In [8]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, FloatSlider, IntSlider
import ipywidgets as widgets
import pandas as pd

style = {'description_width': 'initial'}


R1_Imax_slider = FloatSlider(min=0,max = 5,value= 2, description="R1_Imax", style= style )
R2_Imax_slider = FloatSlider(min=0,max = 5,value= 3, description="R2_Imax", style= style )
R3_Imax_slider = FloatSlider(min=0,max = 5,value= 4, description="R3_Imax", style= style )
R4_Imax_slider = FloatSlider(min=0,max = 5,value= 2, description="R4_Imax", style= style )
R5_Imax_slider = FloatSlider(min=0,max = 5,value= 2, description="R5_Imax", style= style )
R1_R_slider = FloatSlider(min=0,max = 24,value= 8, description="R1_R", style= style )
R2_R_slider = FloatSlider(min=0,max = 24,value= 6, description="R2_R", style= style )
R3_R_slider = FloatSlider(min=0,max = 24,value= 4, description="R3_R", style= style )
R4_R_slider = FloatSlider(min=0,max = 24,value= 10, description="R4_R", style= style )
R5_R_slider = FloatSlider(min=0,max = 24,value= 8, description="R5_R", style= style )

def Rezystory(
    R1_Imax = 2,
    R2_Imax = 3,
    R3_Imax = 4,
    R4_Imax = 2,
    R5_Imax = 2,
    R1_R = 8,
    R2_R = 6,
    R3_R = 4,
    R4_R = 10,
    R5_R = 8): 



  prob1 = LpProblem("Ampere_probleme",LpMaximize)

  I = LpVariable("Prad_Wyjsciowy",0,5,LpContinuous)
  I1 = LpVariable("Prad1",0,None,LpContinuous)
  I2 = LpVariable("Prad2",0,None,LpContinuous)
  I3 = LpVariable("Prad3",0,None,LpContinuous)
  I4 = LpVariable("Prad4",0,None,LpContinuous)
  I5 = LpVariable("Prad5",0,None,LpContinuous)

  prob1+=I,"prad_wyjsciowy"
  prob1+=I1==I*(R2_R/(R1_R+R2_R)),"pradI1"
  prob1+=I2==I*(R1_R/(R1_R+R2_R)),"pradI2"
  prob1+=I3==I1+I2,"pradI3"
  prob1+=I4==I3*(R5_R/(R4_R+R5_R)),"pradI4"
  prob1+=I5==I3*(R4_R/(R4_R+R5_R)),"pradI5"

  prob1+=I1<=R1_Imax,"prad1"
  prob1+=I2<=R2_Imax,"prad2"
  prob1+=I3<=R3_Imax,"prad3"
  prob1+=I4<=R4_Imax,"prad4"
  prob1+=I5<=R5_Imax,"prad5"

  prob1.solve()
  print("Status:", LpStatus[prob1.status])
  for v in prob1.variables():
    print(v.name,"=",v.varValue)



interact(Rezystory,
    R1_Imax = R1_Imax_slider,
    R2_Imax = R2_Imax_slider,
    R3_Imax = R3_Imax_slider,
    R4_Imax = R4_Imax_slider,
    R5_Imax = R5_Imax_slider,
    R1_R = R1_R_slider,
    R2_R = R2_R_slider,
    R3_R = R3_R_slider,
    R4_R = R4_R_slider,
    R5_R = R5_R_slider)

interactive(children=(FloatSlider(value=2.0, description='R1_Imax', max=5.0, style=SliderStyle(description_wid…

<function __main__.Rezystory>

# Moc

             /|\
            / | \
          R1  |   R4
          /   |   \
    ------    R3    ------
          \   |   /
          R2  |  R5 
            \ | /
             \|/

Dla mostka przedstawionego powyżej należy wyznaczyć wartości poszczególnych rezystancji tak aby zminimalizować moc rozpraszaną na rezystorach (na mostku).

|Rezystory|1|2|3|4|5|
|--|--|--|--|--||
|Ui|6|10|4|7|3|
|Ii|4|2|2|2|4|
|dI|1|1|1|1|1|

Ograniczeniami są maksymalne prądy płynące przez rezystory obarczone pewnym przedziałem +-1, tak że:

IR1<=I1+1 && 
IR1>=I1-1

In [4]:
from pulp import *

prob3=LpProblem("Mostek",LpMinimize)

variables = {'I1','I2','I3','I4','I5'}
napiecia = {'I1':6,
            'I2':10,
            'I3':4,
            'I4':7,
            'I5':3}
prady = {'I1':4,
         'I2':2,
         'I3':2,
         'I4':2,
         'I5':4}
delta_prad = 1

zmienne = LpVariable.dicts("Prad",variables,0,None,LpContinuous)

prob3 += lpSum([napiecia[i]*zmienne[i] for i in variables])

for i in variables:
  prob3 += zmienne[i] <= prady[i]+delta_prad
  prob3 += zmienne[i] >= prady[i]-delta_prad

I_result = []
U_result = []

for i in variables:
  U_result.append(napiecia[i])

prob3.solve()
print("Status:", LpStatus[prob3.status])
for v in prob3.variables():
  print(v.name,"=",v.varValue)
  I_result.append(v.varValue)

for i in range(5):
  print("R",i+1," = ",U_result[4-i]/I_result[i]) 

Status: Optimal
Prad_I1 = 3.0
Prad_I2 = 1.0
Prad_I3 = 1.0
Prad_I4 = 1.0
Prad_I5 = 3.0
R 1  =  2.0
R 2  =  10.0
R 3  =  4.0
R 4  =  3.0
R 5  =  2.3333333333333335
